In [5]:
import pymysql
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
ChromeDriverManager().install()
from selenium.webdriver.common.by import By
from datetime import datetime
import re
import time
browser = webdriver.Chrome()

conn=pymysql.connect(
    host='localhost',  # 데이터베이스 서버 주소
    user='root',       # 데이터베이스 사용자 이름
    password='tldkswndlssla',  # 데이터베이스 비밀번호
    db='YES24',       # 데이터베이스 이름
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

with conn.cursor() as cur:


    link_list=[]     
        #got the link of searching "python", and iterated with for statement to get the each page.
        # the goal was get the elements of books to get the links for each book in the next step.
    for page in range(1,4):
        browser.get(f'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber={page}&pageSize=24')
        inform = browser.find_elements(By.CLASS_NAME, 'gd_name')
    # used for statement to iterate each book element to get the each book link and add to list.
    # this will be used to get the informations such as title, publisher, etc in the next step. 

        for book_link in inform:
            link = book_link.get_attribute('href')
            link_list.append(link)
            time.sleep(0.1)

                
    for link in link_list:
        browser.get(link)


        try:
            title = browser.find_element(By.CLASS_NAME, 'gd_name').text
            publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text
            publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
            author = browser.find_element(By.CLASS_NAME, 'gd_auth').text[:-1]
            rating = browser.find_element(By.CLASS_NAME, 'yes_b').text
            rating = float(rating)
            review = browser.find_element(By.CLASS_NAME, 'txC_blue').text
            review=int(review.replace(",",""))
            sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]
            sales=int(sales.replace(",",""))
            price = browser.find_element(By.CLASS_NAME, 'nor_price').text[:-1]
            price=int(price.replace(",",""))    
            ranking = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[0].split(" ")[3][:-1]
            ranking = int(ranking)
            ranking_weeks = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[1].split(" ")[3][:-1]
            ranking_weeks = int(ranking_weeks)
        except Exception as e:
            rating == 0.0
            review == 0
            sales == 0
            ranking == 0
            ranking_weeks == 0
        
            match = re.match(r'(\d+)년 (\d+)월 (\d+)일',publishing)
        if match:
            year, month, day = match.groups()
            data_obj = datetime(int(year), int(month), int(day))
            publishing = data_obj.strftime("%Y-%m-%d")
        else:
            publishing  = "2023-01-01"

        sql = """INSERT INTO BOOKS (title, author, publisher, publishing, rating, 
                                review, sales, price, ranking, ranking_weeks)
                                VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
        cur.execute(sql, (title, author, publisher, publishing))
        conn.commit()


NameError: name 'ranking_weeks' is not defined

In [9]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
ChromeDriverManager().install()
from selenium.webdriver.common.by import By
from datetime import datetime
import re
import time
browser = webdriver.Chrome()



link_list=[]     
    #got the link of searching "python", and iterated with for statement to get the each page.
    # the goal was get the elements of books to get the links for each book in the next step.
for page in range(1,4):
    browser.get(f'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber={page}&pageSize=24')
    inform = browser.find_elements(By.CLASS_NAME, 'gd_name')
# used for statement to iterate each book element to get the each book link and add to list.
# this will be used to get the informations such as title, publisher, etc in the next step. 

    for book_link in inform:
        link = book_link.get_attribute('href')
        link_list.append(link)
        # time.sleep(0.1)

            
for link in link_list:
    browser.get(link)


    try:
        title = browser.find_element(By.CLASS_NAME, 'gd_name').text
        publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text
        publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
        author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
        rating = browser.find_element(By.CLASS_NAME, 'gd_rating').find_element(By.CLASS_NAME, 'yes_b').text
        rating = float(rating.replace(",",""))  
        review = browser.find_element(By.CLASS_NAME, 'gd_reviewCount').find_element(By.CLASS_NAME, 'txC_blue').text
        review=int(review.replace(",",""))
        sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]
        sales=int(sales.replace(",",""))
        price = browser.find_element(By.CLASS_NAME, 'nor_price').text[:-1]
        price=int(price.replace(",",""))    
        ranking = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[0].split(" ")[2][:-1]
        # ranking_weeks = browser.find_element(By.CLASS_NAME, 'gd_best').text
    except Exception as e:
        rating == 0.0
        review == 0
        sales == 0
        ranking == 0
        # ranking_weeks == 0
        

        match = re.match(r'(\d+)년 (\d+)월 (\d+)일',publishing)
        if match:
            year, month, day = match.groups()
            data_obj = datetime(int(year), int(month), int(day))
            publishing = data_obj.strftime("%Y-%m-%d")
        else:
            publishing  = "2023-01-01"


    print(title, author, publisher, publishing, rating, review, sales, price, ranking)
    print(type(rating))


4~7세 보고 만지는 수학은 이렇게 가르칩니다 최경희 블루무스 2024년 04월 30일 10.0 5 153240 22500 1
<class 'float'>
5
나는 읽고 쓰고 버린다 손웅정 난다 2024년 04월 20일 9.7 29 198417 15300 2
<class 'float'>
5
흔한남매 16 흔한남매 원저/백난도 글/유난희 그림/흔한컴퍼니  미래엔아이세움 2024년 04월 25일 10.0 7 138189 13050 3
<class 'float'>
5
불변의 법칙 모건 하우절 저/이수경 서삼독 2024년 02월 28일 9.6 157 507012 22500 4
<class 'float'>
5
나를 소모하지 않는 현명한 태도에 관하여 마티아스 뇔케 저/이미옥 퍼스트펭귄 2024년 03월 10일 9.5 69 327912 16020 5
<class 'float'>
5
삶이 흔들릴 때 뇌과학을 읽습니다 이케가야 유지 글/김성기 힉스 2024년 03월 27일 10.0 27 128163 16200 6
<class 'float'>
5
세이노의 가르침 세이노(SayNo) 데이원 2023년 03월 02일 9.0 2191 1609332 6480 7
<class 'float'>
5
빨모쌤의 라이브 영어회화 신용하 웅진지식하우스 2024년 04월 24일 9.8 44 125049 19800 8
<class 'float'>
5
일류의 조건 사이토 다카시 저/정현 필름 2024년 03월 20일 9.6 217 264108 17820 9
<class 'float'>
5
마이크로 리추얼 : 사소한 것들의 힘 장재열 한국경제신문 2024년 04월 25일 10.0 15 53832 16200 10
<class 'float'>
5
오컬트 3부작 : 장재현 각본집 장재현 유선사 2024-05-16 10.0 15 53832 16200 10
<class 'float'>
5
나는 메트로폴리탄 미술관의 경비원입니다 패트릭 브링리 저/김희정, 조현주 웅진지식하우스 2023년 11월 24일 9.2 28

KeyboardInterrupt: 

In [4]:
for i in range(5):
  i+=1
  print(i)

1
2
3
4
5
